In [1]:
using Rocket
using BenchmarkTools

In [2]:
n = 100000

array      = collect(1:n)
observable = from(array);

In [3]:
reactive(source::S) where S = reactive(as_subscribable(S), source)

function reactive(::ValidSubscribable{T}, source) where T
    keep = KeepActor{T}()
    subscribe!(source, keep)
    return keep.values
end

reactive (generic function with 2 methods)

# Map operator performance comparison

In [4]:
function regularArrayMap(source)
    keep = Vector{Int}()
    for v in source
        push!(keep, v ^ 2)
    end
    return keep
end

observable_mapped = observable |> map(Int, (d) -> d ^ 2)

println(regularArrayMap(array) == reactive(observable_mapped))

@btime regularArrayMap($array);
@btime reactive($observable_mapped);

true
  637.032 μs (17 allocations: 2.00 MiB)
  682.055 μs (19 allocations: 2.00 MiB)


# Filter operator performance comparison

In [5]:
function regularArrayFilter(source)
    keep   = Vector{Int}()
    for v in source
        if v % 2 == 0
            push!(keep, v)
        end
    end
    return keep
end

observable_filtered = observable |> filter((d) -> d % 2 == 0)

println(regularArrayFilter(array) == reactive(observable_filtered))

@btime regularArrayFilter($array);
@btime reactive($observable_filtered);

true
  331.261 μs (16 allocations: 1.00 MiB)
  349.075 μs (18 allocations: 1.00 MiB)


# Count operator performance comparison

In [6]:
function regularArrayCount(source)
    keep    = Vector{Int}()
    current = 0
    for v in source
        current += 1
    end
    push!(keep, current)
    return keep
end

observable_counted = observable |> count()

println(regularArrayCount(array) == reactive(observable_counted))

@btime regularArrayCount($array);
@btime reactive($observable_counted);

true
  54.187 ns (2 allocations: 128 bytes)
  24.007 μs (4 allocations: 176 bytes)


# Enumerate operator performance comparison

In [7]:
function regularArrayEnumerate(source)
    keep   = Vector{Tuple{Int, Int}}()
    current = 1
    for v in source
        push!(keep, (v, current))
        current += 1
    end
    return keep
end

observable_enumerated = observable |> enumerate()

println(regularArrayEnumerate(array) == reactive(observable_enumerated))

@btime regularArrayEnumerate($array);
@btime reactive($observable_enumerated);

true
  1.034 ms (17 allocations: 3.00 MiB)
  1.177 ms (19 allocations: 3.00 MiB)


# Scan operator performance comparison

In [8]:
function regularArrayScan(source, seed::Int)
    keep    = Vector{Int}()
    current = seed
    for v in source
        current = current + v
        push!(keep, current)
    end
    return keep
end

observable_scanned = observable |> scan(Int, +, 0)

println(regularArrayScan(array, 0) == reactive(observable_scanned))

@btime regularArrayScan($array, $0);
@btime reactive($observable_scanned);

true
  689.395 μs (17 allocations: 2.00 MiB)
  734.585 μs (19 allocations: 2.00 MiB)


# Reduce operator performance comparison

In [9]:
function regularArrayReduce(source, seed::Int)
    keep    = Vector{Int}()
    current = seed
    for v in source
        current = current + v
    end
    push!(keep, current)
    return keep
end

observable_reduced = observable |> reduce(Int, +, 0)

println(regularArrayReduce(array, 0) == reactive(observable_reduced))

@btime regularArrayReduce($array, $0);
@btime reactive($observable_reduced);

true
  11.332 μs (2 allocations: 128 bytes)
  33.681 μs (4 allocations: 176 bytes)


# Max operator performance comparison

In [10]:
function regularArrayMax(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v > current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_max = observable |> max()

println(regularArrayMax(array) == reactive(observable_max))

@btime regularArrayMax($array);
@btime reactive($observable_max);

true
  96.314 μs (2 allocations: 128 bytes)
  186.739 μs (4 allocations: 176 bytes)


# Min operator performance comparison

In [11]:
function regularArrayMin(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v < current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_min = observable |> min()

println(regularArrayMin(array) == reactive(observable_min))

@btime regularArrayMin($array);
@btime reactive($observable_min);

true
  199.243 μs (2 allocations: 128 bytes)
  140.125 μs (4 allocations: 176 bytes)


# Sum operator performance comparison

In [12]:
function regularArraySum(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            current = current + v
        end
    end
    push!(keep, current)
    return keep
end

observable_sum = observable |> sum()

println(regularArraySum(array) == reactive(observable_sum))

@btime regularArraySum($array);
@btime reactive($observable_sum);

true
  73.494 μs (2 allocations: 128 bytes)
  164.957 μs (4 allocations: 176 bytes)
